In [1]:
### A1. Membaca data mentah ###
### ----------------------- ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pandas.plotting import radviz

In [2]:
# Source directory and result #
src = r'SampleDataCsv.csv'
df = pd.read_csv(src, sep='|')

str1 = 'AsofDate'
str2 = 'CIF'
str3 = 'Collectibility'
str4 = 'Plafond'
str5 = 'Oustanding'
str6 = 'Business_Type'
str7 = 'OpenDate'
str8 = 'LOB'
str9 = 'DelQ'
str10 = 'Collateral_Type'
str11 = 'Collateral_Value'

df2 = df.groupby([str1, str2]).agg({ str3:'max'
                              ,str4:'max'
                              ,str5:'max'
                              ,str6:'max'
                              ,str7:'max'
                              ,str8:'max'
                              ,str9:'max'
                              ,str10:'max'
                              ,str11:'max'})[[str3
                                             ,str4
                                             ,str5
                                             ,str6
                                             ,str7
                                             ,str8
                                             ,str9
                                             ,str10
                                             ,str11]].reset_index()

In [3]:
# print(df2)

## Additional coloumn ##
df2["Sisa_Plafond"] = df2[str4] - df2[str5]

In [4]:
## Bucket Dpd - range mapping value ## 
bins = [0, 1, 30, 60, 90, 120, 150, 180]
names = ['1_Current', '2_DPD1+', '3_DPD30+', '4_DPD60+', 
         '5_DPD90+', '6_DPD120+', '7_DPD150+', '8_DPD180+']
d = dict(enumerate(names, 1))
df2['Bucket'] = np.vectorize(d.get)(np.digitize(df2[str9], bins))

In [5]:
## Kolektibitas mapping ## 
bins = [1, 2, 5]
names = ['1_Current', '2_SM', '3_NPL']
d = dict(enumerate(names, 1))
df2['Collectibility_Desc'] = np.vectorize(d.get)(np.digitize(df2[str3], bins))

In [6]:
## Oustanding ##
bins = [0, 50000000, 80000000, 100000000, 250000000, 350000000]
names = ['A_0-50 Jt', 'B_>50-80 Jt', 'C_>80-100 Jt', 
         'D_>100-250 Jt', 'E_>250-350 Jt', 'F_>350 Jt']
d = dict(enumerate(names, 1))
df2['Oustanding_Band'] = np.vectorize(d.get)(np.digitize(df2[str5], bins))

In [7]:
## Temporary ##
Collectibility = df2['Collectibility'].astype(int)
Collateral_Value = df2['Collateral_Value'].astype(float)
OS = df2["Oustanding"].astype(int)

In [8]:
# df2['Collateral_remains'] = OS - Collateral_Value
df2.loc[(Collateral_Value < OS), 'Collateral_remains'] = OS - Collateral_Value
df2.loc[(Collateral_Value > OS), 'Collateral_remains'] = 0
df2.loc[(Collateral_Value == OS), 'Collateral_remains'] = 0
df2.dropna(inplace=True)

In [9]:
## Grouping tipe bisnis ##
Business_Type = df2['Business_Type'].astype(int)
df2.loc[(Business_Type.isin([9990,1019,6000,1007])), 'Type_Buss'] = Business_Type
df2.loc[(Business_Type.isin([1001,1004,1008,1015,1020,8000,9000,1014,1016])), 'Type_Buss'] = 'Other'
df2.loc[(Business_Type > 150000), 'Type_Buss'] = 'Other'

In [10]:
## Create Debtor current month ##
DirResult = "C:\Data\BTN_Consolidate_2.csv"
df2.to_csv( DirResult, index = False, sep='|')
df_rawdata = df2
# print(df_rawdata.head(3))
print('Membuat dataset rawdata - selesai')

Membuat dataset rawdata - selesai


In [11]:
### A2. Membuat dataset kualitas pinjaman ###
### ------------------------------------- ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import poisson

In [12]:
## Read source data ##
DirSource = "C:\Data\BTN_Consolidate_2.csv"
df_Quality = pd.read_csv(DirSource, sep='|')

In [13]:
# Check null value #
# df_Quality.isnull().sum()

df_Quality['AsofDate'] = pd.to_datetime(df_Quality["AsofDate"]).dt.strftime('%m-%Y')
df_Quality["OS"] = (df_Quality['Oustanding']/1000000).astype(int)

df_Quality = df_Quality.groupby(['AsofDate','Collectibility'])['OS'].sum().unstack(fill_value=0)
df_Quality.reset_index(inplace=True)

df_Quality = pd.DataFrame(df_Quality,columns=['AsofDate',1,2,3,4,5,'OS','CUR','SM','NPL','NPL+'])
df_Quality.replace(np.NaN, 0, inplace=True)
df_Quality.astype(str).replace('\.0', '', regex=True)

,AsofDate,1,2,3,4,5,OS,CUR,SM,NPL,NPL+
0,01-2019,1448,0,0,0,0,0,0,0,0,0
1,02-2019,931,0,0,0,0,0,0,0,0,0
2,03-2019,953,0,0,0,0,0,0,0,0,0
3,04-2019,658,0,0,0,0,0,0,0,0,0
4,05-2019,636,0,0,0,0,0,0,0,0,0
5,06-2019,215,0,0,0,0,0,0,0,0,0
6,07-2019,603,0,0,0,0,0,0,0,0,0
7,08-2019,589,0,0,0,0,0,0,0,0,0
8,09-2019,21,0,0,0,0,0,0,0,0,0
9,10-2019,255,0,0,0,0,0,0,0,0,0


In [14]:
# df_Quality.isnull().sum()

df_Quality['OS'] = (df_Quality[1] + df_Quality[2]+ df_Quality[3]+ df_Quality[4]+ df_Quality[5]).astype(int)
df_Quality['CUR'] = (df_Quality[1]).astype(int)
df_Quality['SM'] = (df_Quality[2]).astype(int)
df_Quality['NPL'] = (df_Quality[3] + df_Quality[4] + df_Quality[5]).astype(int)
df_Quality['NPL+'] = (df_Quality[2] + df_Quality[3] + df_Quality[4] + df_Quality[5]).astype(int)

df_Quality['%Curr'] = (df_Quality[1]/df_Quality['OS']*100).round(1)
df_Quality['%SM'] = (df_Quality['SM']/df_Quality['OS']*100).round(1)
df_Quality['%NPL'] = (df_Quality['NPL']/df_Quality['OS']*100).round(1)
df_Quality['%NPL+'] = (df_Quality['NPL+']/df_Quality['OS']*100).round(1)

df_Quality.reset_index(drop=True, inplace=True)

In [15]:
# export to Csv ##
DirResult = (r"C:\Data\Result\BTN_Forcasting.csv")
df_Quality.to_csv( DirResult, index = False, sep='|')
# print(df_Quality)
print('Membuat dataset kualitas pinjaman - selesai')

Membuat dataset kualitas pinjaman - selesai


In [ ]:
# Source directory and result #
src = r'C:\Data\SampleDataCsv.csv'
df = pd.read_csv(src, sep='|')

str1 = 'AsofDate'
str2 = 'CIF'
str3 = 'Collectibility'
str4 = 'Plafond'
str5 = 'Oustanding'
str6 = 'Business_Type'
str7 = 'OpenDate'
str8 = 'LOB'
str9 = 'DelQ'
str10 = 'Collateral_Type'
str11 = 'Collateral_Value'

df2 = df.groupby([str2]).agg({ str3:'max'
                              ,str4:'max'
                              ,str5:'max'
                              ,str6:'max'
                              ,str7:'max'
                              ,str8:'max'
                              ,str9:'max'
                              ,str10:'max'
                              ,str11:'max'})[[str3
                                             ,str4
                                             ,str5
                                             ,str6
                                             ,str7
                                             ,str8
                                             ,str9
                                             ,str10
                                             ,str11]].reset_index()

# print(df2)